<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/Automated_Financial_News_Analysis_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install -q requests beautifulsoup4 pandas gradio google-generativeai gspread oauth2client scikit-learn jieba feedparser wordcloud matplotlib plotly kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.4 MB/s eta 0:00:00


In [13]:

# ==================== 匯入套件 ====================
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime, timedelta
import time
import gradio as gr
from google.colab import auth
import gspread
from google.auth import default
import google.generativeai as genai
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import jieba.analyse
from collections import Counter
import re
import feedparser


In [14]:

# ==================== Google Sheets 設定 ====================
def setup_google_sheets():
    """設定 Google Sheets 連線"""
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    return gc


In [15]:

# ==================== Gemini API 設定 ====================
def setup_gemini(api_key):
    """設定 Gemini API"""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('models/gemini-2.5-pro')
    return model


In [16]:

# ==================== 財經新聞爬蟲（改良版）====================
class FinanceNewsCrawler:
    """爬取台股財經新聞 - 使用多種穩定來源"""

    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        }
        self.news_data = []

    def crawl_udn_rss(self, max_news=30):
        """方法1: 爬取聯合新聞網 RSS（最穩定）"""
        self.news_data = []

        try:
            print("🔍 正在爬取聯合新聞網 RSS...")
            rss_urls = [
                'https://udn.com/rssfeed/news/2/6644?ch=news',  # 股市要聞
                'https://udn.com/rssfeed/news/2/6645?ch=news',  # 上市電子
            ]

            for rss_url in rss_urls:
                try:
                    feed = feedparser.parse(rss_url)

                    for entry in feed.entries[:15]:
                        self.news_data.append({
                            '標題': entry.title,
                            '摘要': entry.summary if hasattr(entry, 'summary') else '',
                            '連結': entry.link,
                            '發布時間': entry.published if hasattr(entry, 'published') else '未知時間',
                            '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                            '來源': '聯合新聞網'
                        })

                        if len(self.news_data) >= max_news:
                            break

                    time.sleep(1)
                except Exception as e:
                    print(f"⚠️ RSS 爬取部分失敗: {str(e)}")
                    continue

            print(f"✅ 成功爬取 {len(self.news_data)} 則新聞（聯合新聞網）")
            return self.news_data

        except Exception as e:
            print(f"❌ RSS 爬取失敗: {str(e)}")
            return []

    def crawl_ctee_news(self, max_news=30):
        """方法2: 爬取工商時報（備用方案）"""
        self.news_data = []

        try:
            print("🔍 正在爬取工商時報...")
            url = "https://ctee.com.tw/news/stock"

            response = requests.get(url, headers=self.headers, timeout=15)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')

            # 找到新聞列表
            news_items = soup.find_all('div', class_='item')

            for item in news_items[:max_news]:
                try:
                    title_tag = item.find('h3')
                    link_tag = item.find('a')
                    time_tag = item.find('time')

                    if title_tag and link_tag:
                        title = title_tag.get_text(strip=True)
                        link = link_tag['href'] if link_tag['href'].startswith('http') else 'https://ctee.com.tw' + link_tag['href']
                        publish_time = time_tag.get_text(strip=True) if time_tag else '未知時間'

                        self.news_data.append({
                            '標題': title,
                            '摘要': '',
                            '連結': link,
                            '發布時間': publish_time,
                            '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                            '來源': '工商時報'
                        })
                except Exception as e:
                    continue

            print(f"✅ 成功爬取 {len(self.news_data)} 則新聞（工商時報）")
            return self.news_data

        except Exception as e:
            print(f"❌ 工商時報爬取失敗: {str(e)}")
            return []

    def crawl_demo_data(self):
        """方法3: 使用示範資料（保證可執行）"""
        print("🔍 使用示範新聞資料...")

        self.news_data = [
            {'標題': '台積電法說會釋利多 外資目標價上看1200元', '摘要': '台積電舉行法說會，釋出多項利多消息，外資看好後市', '連結': 'https://example.com/1', '發布時間': '2025-10-26 10:30', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '美股科技股大漲 台股電子股可望受惠', '摘要': '美國科技股強勁上漲，帶動台灣電子股買氣', '連結': 'https://example.com/2', '發布時間': '2025-10-26 09:15', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': 'AI需求強勁 半導體產業鏈營收創新高', '摘要': '人工智慧應用帶動半導體需求大增', '連結': 'https://example.com/3', '發布時間': '2025-10-26 08:45', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '台股站穩萬八 外資連續買超', '摘要': '台灣股市表現強勁，外資持續買入', '連結': 'https://example.com/4', '發布時間': '2025-10-25 16:20', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '聯發科推出新款5G晶片 獲市場好評', '摘要': '聯發科最新晶片效能提升，預期營收成長', '連結': 'https://example.com/5', '發布時間': '2025-10-25 14:30', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '鴻海電動車佈局加速 訂單能見度提升', '摘要': '鴻海在電動車領域持續擴展，獲得多筆訂單', '連結': 'https://example.com/6', '發布時間': '2025-10-25 11:00', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '金融股配息題材發酵 吸引存股族', '摘要': '金融股進入除權息旺季，殖利率吸引人', '連結': 'https://example.com/7', '發布時間': '2025-10-24 15:45', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '台股基金單週吸金破百億 投資人搶進', '摘要': '台股基金受到投資人青睞，資金大量湧入', '連結': 'https://example.com/8', '發布時間': '2025-10-24 13:20', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '航運股受油價影響 股價震盪', '摘要': '國際油價波動，影響航運業成本', '連結': 'https://example.com/9', '發布時間': '2025-10-24 10:10', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '生技股獲政府補助 題材發酵', '摘要': '政府加碼生技產業補助，相關個股受惠', '連結': 'https://example.com/10', '發布時間': '2025-10-23 16:50', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '台幣升值壓力 出口產業受關注', '摘要': '新台幣匯率走強，出口企業面臨挑戰', '連結': 'https://example.com/11', '發布時間': '2025-10-23 14:25', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '電動車供應鏈夯 相關個股漲勢凌厲', '摘要': '電動車產業鏈持續成長，帶動股價', '連結': 'https://example.com/12', '發布時間': '2025-10-23 11:30', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '記憶體價格回升 相關廠商營收看增', '摘要': 'DRAM與NAND價格上漲，記憶體廠受惠', '連結': 'https://example.com/13', '發布時間': '2025-10-22 15:15', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '被動元件缺貨 廠商調漲報價', '摘要': '被動元件供不應求，價格持續上漲', '連結': 'https://example.com/14', '發布時間': '2025-10-22 12:40', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '5G基建投資加速 電信股受矚目', '摘要': '各國加速5G建設，電信業迎來商機', '連結': 'https://example.com/15', '發布時間': '2025-10-22 09:55', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '綠能政策推動 太陽能概念股發光', '摘要': '政府綠能政策明確，太陽能產業受惠', '連結': 'https://example.com/16', '發布時間': '2025-10-21 16:30', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '晶圓代工產能滿載 業者擴廠加速', '摘要': '晶圓代工需求旺盛，廠商積極擴產', '連結': 'https://example.com/17', '發布時間': '2025-10-21 13:45', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': 'PCB產業需求回溫 法人喊買', '摘要': '印刷電路板需求復甦，法人看好後市', '連結': 'https://example.com/18', '發布時間': '2025-10-21 10:20', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '台股權值股輪動 中小型股活躍', '摘要': '大盤輪動效應明顯，中小型股交易熱絡', '連結': 'https://example.com/19', '發布時間': '2025-10-20 15:10', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
            {'標題': '科技業人才荒 各公司加薪搶人', '摘要': '科技產業缺工嚴重，企業提高薪資', '連結': 'https://example.com/20', '發布時間': '2025-10-20 11:55', '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '來源': '示範資料'},
        ]

        print(f"✅ 載入 {len(self.news_data)} 則示範新聞")
        return self.news_data

    def auto_crawl(self, max_news=30):
        """自動選擇最佳爬蟲方法"""
        print("🚀 開始智慧爬蟲...")

        # 優先順序：RSS > 工商時報 > 示範資料
        methods = [
            ('RSS Feed', self.crawl_udn_rss),
            ('工商時報', self.crawl_ctee_news),
            ('示範資料', self.crawl_demo_data)
        ]

        for method_name, method in methods:
            try:
                print(f"📡 嘗試使用: {method_name}")
                if method_name == '示範資料':
                    news = method()
                else:
                    news = method(max_news)

                if news and len(news) > 0:
                    print(f"✅ 成功！使用 {method_name} 取得 {len(news)} 則新聞")
                    return news
            except Exception as e:
                print(f"⚠️ {method_name} 失敗，嘗試下一個方法...")
                continue

        print("❌ 所有方法都失敗")
        return []

In [5]:

# ==================== 財經新聞爬蟲 ====================
class FinanceNewsCrawler:
    """爬取台股財經新聞"""

    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        self.news_data = []

    def crawl_cnyes_news(self, pages=3):
        """爬取鉅亨網台股新聞"""
        self.news_data = []
        base_url = "https://news.cnyes.com/news/cat/tw_stock"

        for page in range(1, pages + 1):
            try:
                print(f"🔍 正在爬取第 {page}/{pages} 頁...")
                url = f"{base_url}?page={page}"
                response = requests.get(url, headers=self.headers, timeout=10)
                response.encoding = 'utf-8'
                soup = BeautifulSoup(response.text, 'html.parser')

                # 找到新聞列表
                news_items = soup.find_all('div', class_='_1Zdp')

                for item in news_items:
                    try:
                        # 標題
                        title_tag = item.find('a')
                        if not title_tag:
                            continue

                        title = title_tag.get_text(strip=True)
                        link = "https://news.cnyes.com" + title_tag['href']

                        # 時間
                        time_tag = item.find('time')
                        publish_time = time_tag.get_text(strip=True) if time_tag else "未知時間"

                        # 摘要
                        summary_tag = item.find('p')
                        summary = summary_tag.get_text(strip=True) if summary_tag else ""

                        self.news_data.append({
                            '標題': title,
                            '摘要': summary,
                            '連結': link,
                            '發布時間': publish_time,
                            '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        })
                    except Exception as e:
                        continue

                time.sleep(2)  # 避免請求太快

            except Exception as e:
                print(f"❌ 第 {page} 頁爬取失敗: {str(e)}")
                continue

        print(f"✅ 成功爬取 {len(self.news_data)} 則新聞")
        return self.news_data

    def crawl_yahoo_finance_news(self, pages=3):
        """爬取 Yahoo 財經台股新聞"""
        self.news_data = []

        for page in range(pages):
            try:
                print(f"🔍 正在爬取第 {page+1}/{pages} 頁...")
                url = f"https://tw.stock.yahoo.com/news/list?category=%E5%8F%B0%E8%82%A1&offset={page*10}"
                response = requests.get(url, headers=self.headers, timeout=10)
                response.encoding = 'utf-8'
                soup = BeautifulSoup(response.text, 'html.parser')

                # 找到新聞列表
                news_items = soup.find_all('div', class_='Ov(h)')

                for item in news_items:
                    try:
                        title_tag = item.find('a')
                        if not title_tag:
                            continue

                        title = title_tag.get_text(strip=True)
                        link = "https://tw.stock.yahoo.com" + title_tag['href']

                        # 時間
                        time_tag = item.find('time')
                        publish_time = time_tag.get_text(strip=True) if time_tag else "未知時間"

                        self.news_data.append({
                            '標題': title,
                            '摘要': '',
                            '連結': link,
                            '發布時間': publish_time,
                            '爬取時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        })
                    except Exception as e:
                        continue

                time.sleep(2)

            except Exception as e:
                print(f"❌ 第 {page+1} 頁爬取失敗: {str(e)}")
                continue

        print(f"✅ 成功爬取 {len(self.news_data)} 則新聞")
        return self.news_data

In [29]:
# ==================== 文字分析（完全修正版）====================
class TextAnalyzer:
    """文字分析與關鍵字提取 - 結巴分詞增強版"""

    def __init__(self):
        print("🔧 初始化結巴分詞...")

        # 載入結巴分詞
        import jieba
        import jieba.analyse

        # 擴充停用詞列表（包含網址相關詞）
        self.stopwords = set([
            # 基本停用詞
            '的', '是', '在', '了', '和', '與', '等', '將', '可', '或',
            '但', '對', '為', '及', '以', '於', '從', '更', '很', '最',
            '也', '都', '就', '到', '被', '有', '這', '那', '個', '他',
            '她', '我', '你', '們', '中', '上', '下', '來', '去', '說',
            '要', '會', '能', '把', '讓', '給', '沒', '只', '還', '又',
            '則', '篇', '位', '家', '間', '名', '次', '項', '件', '張',
            '不', '其', '而', '因', '所', '則', '得', '著', '過', '便',
            '用', '並', '向', '如', '且', '再', '另', '即', '或許', '此',
            '表示', '指出', '認為', '根據', '透過', '目前', '今年', '去年',
            # 網址相關停用詞（關鍵！）
            'http', 'https', 'www', 'com', 'tw', 'net', 'org', 'url',
            'amp', 'photo', 'image', 'jpg', 'png', 'html', 'php',
            'udn', 'news', 'article', 'id', 'link', 'href',
            # 數字相關
            '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            '10', '20', '30', '100', '180', '360', '720', '1080',
            '3600', '2024', '2025', '2026',
            # 時間相關通用詞
            '今天', '昨天', '明天', '上午', '下午', '晚上'
        ])

        # 自訂詞典（金融相關詞彙）
        financial_terms = [
            # 公司名稱
            '台積電', '聯發科', '鴻海', '大立光', '台達電', '聯電',
            '日月光', '國巨', '華碩', '廣達', '仁寶', '和碩',
            '台塑', '中鋼', '富邦金', '國泰金', '兆豐金', '玉山金',
            # 金融術語
            '外資', '投信', '自營商', '三大法人', '散戶',
            '漲停', '跌停', '漲跌幅', '成交量', '融資', '融券',
            '本益比', '殖利率', '股息', '配息', '除權息',
            '現金股利', '股票股利', '填權息',
            # 產業類別
            '半導體', '電子股', '金融股', '傳產股', '生技股',
            '航運股', '鋼鐵股', '塑化股', '營建股', '觀光股',
            # 財務指標
            '營收', '獲利', '財報', 'EPS', 'ROE', 'ROA',
            '毛利率', '營益率', '淨利率', '負債比',
            # 市場相關
            '美股', '台股', '陸股', '港股', '日股',
            '加權指數', '櫃買指數', '道瓊', '那斯達克', 'S&P',
            '多頭', '空頭', '盤整', '突破', '支撐', '壓力',
            # 總經相關
            '升息', '降息', '通膨', '通縮', '經濟成長', 'GDP',
            '央行', '聯準會', '貨幣政策', '財政政策',
            # 科技趨勢
            'AI', '人工智慧', '電動車', '5G', '物聯網',
            '元宇宙', '區塊鏈', '雲端', '大數據',
            # 其他
            '投資', '買進', '賣出', '持有', '布局', '獲利了結'
        ]

        # 將自訂詞加入結巴詞典
        for term in financial_terms:
            jieba.add_word(term, freq=10000)  # 設定高頻率確保被識別

        print("✅ 結巴分詞初始化完成")
        print(f"📚 停用詞數量: {len(self.stopwords)} 個")
        print(f"📖 自訂詞彙數量: {len(financial_terms)} 個")

    def preprocess_text(self, text):
        """文字預處理 - 清除網址和雜訊"""
        # 1. 移除網址
        text = re.sub(r'https?://\S+|www\.\S+', '', text)

        # 2. 移除 email
        text = re.sub(r'\S+@\S+', '', text)

        # 3. 移除數字（但保留中文數字）
        text = re.sub(r'\d+', '', text)

        # 4. 移除特殊符號（保留中文、英文）
        text = re.sub(r'[^\w\s\u4e00-\u9fff]', ' ', text)

        # 5. 移除多餘空白
        text = ' '.join(text.split())

        return text

    def is_valid_keyword(self, word):
        """判斷是否為有效關鍵字"""
        # 檢查長度
        if len(word) < 2:
            return False

        # 檢查是否為停用詞
        if word.lower() in self.stopwords:
            return False

        # 檢查是否全為數字
        if word.isdigit():
            return False

        # 檢查是否為純英文且太短
        if word.isalpha() and len(word) < 3:
            return False

        # 檢查是否包含網址關鍵字
        url_keywords = ['http', 'www', 'com', 'tw', 'net', 'html', 'php', 'amp']
        if any(uk in word.lower() for uk in url_keywords):
            return False

        return True

    def extract_keywords_jieba(self, texts, top_n=10):
        """使用 jieba.analyse 提取關鍵字"""
        print("🔍 使用 jieba.analyse 提取關鍵字...")

        # 預處理並合併所有文字
        processed_texts = [self.preprocess_text(text) for text in texts]
        combined_text = ' '.join(processed_texts)

        # 使用 jieba 的 TF-IDF 提取關鍵字
        keywords = jieba.analyse.extract_tags(
            combined_text,
            topK=top_n * 3,  # 提取更多，之後再過濾
            withWeight=True,
            allowPOS=('n', 'nr', 'ns', 'nt', 'nz', 'v', 'vn', 'a', 'an')
        )

        # 過濾無效關鍵字
        valid_keywords = [
            (word, weight) for word, weight in keywords
            if self.is_valid_keyword(word)
        ]

        print(f"✅ 提取了 {len(valid_keywords[:top_n])} 個有效關鍵字")
        return valid_keywords[:top_n]

    def extract_keywords_textrank(self, texts, top_n=10):
        """使用 TextRank 演算法提取關鍵字"""
        print("🔍 使用 TextRank 演算法提取關鍵字...")

        # 預處理並合併所有文字
        processed_texts = [self.preprocess_text(text) for text in texts]
        combined_text = ' '.join(processed_texts)

        # 使用 jieba 的 TextRank 提取關鍵字
        keywords = jieba.analyse.textrank(
            combined_text,
            topK=top_n * 3,
            withWeight=True,
            allowPOS=('n', 'nr', 'ns', 'nt', 'nz', 'v', 'vn', 'a', 'an')
        )

        # 過濾無效關鍵字
        valid_keywords = [
            (word, weight) for word, weight in keywords
            if self.is_valid_keyword(word)
        ]

        print(f"✅ 提取了 {len(valid_keywords[:top_n])} 個有效關鍵字")
        return valid_keywords[:top_n]

    def word_frequency(self, texts, top_n=10):
        """詞頻統計"""
        print("🔍 進行詞頻統計...")

        # 預處理並合併所有文字
        processed_texts = [self.preprocess_text(text) for text in texts]
        combined_text = ' '.join(processed_texts)

        # 使用結巴分詞
        words = jieba.cut(combined_text)

        # 過濾並統計
        valid_words = [
            w.strip() for w in words
            if w.strip() and self.is_valid_keyword(w.strip())
        ]

        # 計算詞頻
        word_counts = Counter(valid_words)

        result = word_counts.most_common(top_n)
        print(f"✅ 統計了 {len(word_counts)} 個不同的詞，返回前 {top_n} 個")

        return result

    def tfidf_analysis(self, texts, top_n=10):
        """使用 sklearn TF-IDF 分析"""
        print("🔍 使用 TF-IDF 分析關鍵字...")

        # 中文分詞函數
        def tokenize(text):
            text = self.preprocess_text(text)
            words = jieba.cut(text)
            valid = [w.strip() for w in words if w.strip() and self.is_valid_keyword(w.strip())]
            return ' '.join(valid)

        # 對所有文字進行分詞
        tokenized_texts = [tokenize(text) for text in texts]

        # 過濾空文本
        tokenized_texts = [t for t in tokenized_texts if t.strip()]

        if not tokenized_texts:
            print("⚠️ 沒有有效文本，改用 jieba 方法")
            return self.extract_keywords_jieba(texts, top_n)

        try:
            # 建立 TF-IDF 向量化器
            vectorizer = TfidfVectorizer(
                max_features=top_n * 3,
                token_pattern=r'(?u)\b\w\w+\b',  # 至少2個字符
                min_df=1,  # 至少出現在1個文檔中
                max_df=0.8  # 最多出現在80%的文檔中
            )

            # 計算 TF-IDF 矩陣
            tfidf_matrix = vectorizer.fit_transform(tokenized_texts)

            # 取得特徵名稱
            feature_names = vectorizer.get_feature_names_out()

            # 計算每個關鍵字的總分數
            tfidf_scores = tfidf_matrix.sum(axis=0).A1

            # 建立配對
            keywords = list(zip(feature_names, tfidf_scores))

            # 再次過濾（雙重保險）
            keywords = [(w, s) for w, s in keywords if self.is_valid_keyword(w)]

            # 排序
            keywords.sort(key=lambda x: x[1], reverse=True)

            print(f"✅ TF-IDF 分析完成，提取 {min(top_n, len(keywords))} 個關鍵字")

            return keywords[:top_n]

        except Exception as e:
            print(f"⚠️ TF-IDF 分析失敗: {str(e)}")
            print("🔄 改用 jieba 關鍵字提取...")
            return self.extract_keywords_jieba(texts, top_n)

    def comprehensive_analysis(self, texts, top_n=10):
        """綜合分析"""
        print("\n" + "="*60)
        print("🚀 開始綜合文字分析（已過濾網址和雜訊）")
        print("="*60)

        results = {
            'word_frequency': [],
            'jieba_tfidf': [],
            'textrank': [],
            'sklearn_tfidf': []
        }

        try:
            print("\n📊 方法1: 詞頻統計")
            results['word_frequency'] = self.word_frequency(texts, top_n)
            self._print_keywords(results['word_frequency'], '詞頻統計')
        except Exception as e:
            print(f"❌ 詞頻統計失敗: {str(e)}")

        try:
            print("\n📊 方法2: jieba TF-IDF")
            results['jieba_tfidf'] = self.extract_keywords_jieba(texts, top_n)
            self._print_keywords(results['jieba_tfidf'], 'jieba TF-IDF')
        except Exception as e:
            print(f"❌ jieba TF-IDF 失敗: {str(e)}")

        try:
            print("\n📊 方法3: TextRank 演算法")
            results['textrank'] = self.extract_keywords_textrank(texts, top_n)
            self._print_keywords(results['textrank'], 'TextRank')
        except Exception as e:
            print(f"❌ TextRank 失敗: {str(e)}")

        try:
            print("\n📊 方法4: sklearn TF-IDF")
            results['sklearn_tfidf'] = self.tfidf_analysis(texts, top_n)
            self._print_keywords(results['sklearn_tfidf'], 'sklearn TF-IDF')
        except Exception as e:
            print(f"❌ sklearn TF-IDF 失敗: {str(e)}")

        print("\n" + "="*60)
        print("✅ 綜合分析完成")
        print("="*60 + "\n")

        return results

    def _print_keywords(self, keywords, method_name):
        """輔助函數：印出關鍵字"""
        if keywords:
            print(f"\n【{method_name}】前 10 個關鍵字:")
            for idx, (word, score) in enumerate(keywords[:10], 1):
                print(f"  {idx}. {word} ({score:.4f})")
        else:
            print(f"【{method_name}】無結果")

    def get_best_keywords(self, texts, top_n=10, method='tfidf'):
        """取得最佳關鍵字"""
        if method == 'tfidf':
            return self.tfidf_analysis(texts, top_n)
        elif method == 'textrank':
            return self.extract_keywords_textrank(texts, top_n)
        elif method == 'frequency':
            return self.word_frequency(texts, top_n)
        elif method == 'comprehensive':
            # 綜合多種方法
            jieba_kw = dict(self.extract_keywords_jieba(texts, top_n * 2))
            textrank_kw = dict(self.extract_keywords_textrank(texts, top_n * 2))

            all_keywords = set(jieba_kw.keys()) | set(textrank_kw.keys())
            combined_scores = {}

            for kw in all_keywords:
                if self.is_valid_keyword(kw):  # 再次確認
                    score = 0
                    if kw in jieba_kw:
                        score += jieba_kw[kw]
                    if kw in textrank_kw:
                        score += textrank_kw[kw]
                    combined_scores[kw] = score

            sorted_kw = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
            return sorted_kw[:top_n]
        else:
            return self.tfidf_analysis(texts, top_n)

In [41]:
# ==================== 修正後的視覺化模組 ====================
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from PIL import Image
import numpy as np

class DataVisualizer:
    """資料視覺化類別 - 使用 Plotly"""

    def __init__(self):
        self.colors = px.colors.qualitative.Set3

    def create_wordcloud(self, keywords, title="關鍵字文字雲", max_words=50):
        """
        生成文字雲圖 - 修正版（返回PIL Image）
        keywords: [(詞, 分數), ...] 格式
        """
        print(f"🎨 生成文字雲: {title}")

        try:
            # 準備文字雲資料
            word_freq = {word: float(score) for word, score in keywords[:max_words]}

            if not word_freq:
                print("⚠️ 沒有關鍵字資料")
                return None

            # 生成文字雲
            wordcloud = WordCloud(
                width=1600,
                height=800,
                background_color='white',
                font_path='/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',  # 中文字體
                colormap='viridis',
                max_words=max_words,
                relative_scaling=0.5,
                min_font_size=12,
                prefer_horizontal=0.7,
                collocations=False  # 避免重複詞組
            ).generate_from_frequencies(word_freq)

            # 轉換為 numpy array
            wordcloud_array = wordcloud.to_array()

            # 轉換為 PIL Image
            wordcloud_image = Image.fromarray(wordcloud_array)

            # 儲存為臨時檔案
            temp_path = '/tmp/wordcloud.png'
            wordcloud_image.save(temp_path, format='PNG', dpi=(300, 300))

            print(f"✅ 文字雲生成完成 ({len(word_freq)} 個詞)")
            print(f"📁 儲存路徑: {temp_path}")

            return temp_path  # 返回檔案路徑

        except Exception as e:
            print(f"❌ 文字雲生成失敗: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def create_wordcloud_plotly(self, keywords, title="關鍵字文字雲", max_words=50):
        """
        使用 Plotly 製作文字雲替代方案（散點圖模擬）
        """
        print(f"🎨 生成 Plotly 文字雲: {title}")

        try:
            top_keywords = keywords[:max_words]

            if not top_keywords:
                print("⚠️ 沒有關鍵字資料")
                return None

            words = [kw for kw, _ in top_keywords]
            scores = [float(score) for _, score in top_keywords]

            # 正規化分數
            max_score = max(scores)
            min_score = min(scores)

            # 計算字體大小（10-60）
            sizes = [10 + (s - min_score) / (max_score - min_score) * 50 for s in scores]

            # 隨機但固定的位置
            import random
            random.seed(42)

            # 使用螺旋佈局
            angles = [i * 137.5 for i in range(len(words))]  # 黃金角度
            radii = [i ** 0.5 for i in range(len(words))]

            x_coords = [r * np.cos(np.radians(a)) for r, a in zip(radii, angles)]
            y_coords = [r * np.sin(np.radians(a)) for r, a in zip(radii, angles)]

            # 建立散點圖
            fig = go.Figure()

            for i, (word, x, y, size, score) in enumerate(zip(words, x_coords, y_coords, sizes, scores)):
                fig.add_trace(go.Scatter(
                    x=[x],
                    y=[y],
                    mode='text',
                    text=word,
                    textfont=dict(
                        size=size,
                        color=f'hsl({i * 360 / len(words)}, 70%, 50%)'
                    ),
                    hovertemplate=f'<b>{word}</b><br>分數: {score:.4f}<extra></extra>',
                    showlegend=False
                ))

            fig.update_layout(
                title=dict(
                    text=f'<b>{title}</b>',
                    x=0.5,
                    xanchor='center',
                    font=dict(size=24)
                ),
                xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
                yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
                height=800,
                template='plotly_white',
                hovermode='closest',
                plot_bgcolor='rgba(240, 240, 240, 0.5)'
            )

            print(f"✅ Plotly 文字雲生成完成")
            return fig

        except Exception as e:
            print(f"❌ Plotly 文字雲生成失敗: {str(e)}")
            return None

    def plot_top_keywords_bar(self, keywords, title="前20關鍵字排行", top_n=20):
        """關鍵字橫條圖（Plotly）"""
        print(f"📊 生成關鍵字橫條圖: {title}")

        try:
            top_keywords = keywords[:top_n]

            if not top_keywords:
                print("⚠️ 沒有關鍵字資料")
                return None

            # 反轉順序
            words = [kw for kw, _ in reversed(top_keywords)]
            scores = [float(score) for _, score in reversed(top_keywords)]

            # 建立橫條圖
            fig = go.Figure(go.Bar(
                x=scores,
                y=words,
                orientation='h',
                marker=dict(
                    color=scores,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title="分數")
                ),
                text=[f'{s:.4f}' for s in scores],
                textposition='outside',
                hovertemplate='<b>%{y}</b><br>分數: %{x:.4f}<extra></extra>'
            ))

            fig.update_layout(
                title=dict(
                    text=f'<b>{title}</b>',
                    x=0.5,
                    xanchor='center',
                    font=dict(size=20)
                ),
                xaxis_title='TF-IDF 分數',
                yaxis_title='關鍵字',
                height=max(500, top_n * 30),
                showlegend=False,
                template='plotly_white',
                margin=dict(l=120, r=50, t=80, b=50)
            )

            print(f"✅ 橫條圖生成完成 ({len(top_keywords)} 個關鍵字)")
            return fig

        except Exception as e:
            print(f"❌ 橫條圖生成失敗: {str(e)}")
            return None

    # ... 其他視覺化方法保持不變 ...

    def plot_keyword_distribution_pie(self, keywords, title="關鍵字分數分布", top_n=15):
        """關鍵字圓餅圖"""
        print(f"📊 生成關鍵字圓餅圖: {title}")

        try:
            top_keywords = keywords[:top_n]

            if not top_keywords:
                print("⚠️ 沒有關鍵字資料")
                return None

            words = [kw for kw, _ in top_keywords]
            scores = [float(score) for _, score in top_keywords]

            fig = go.Figure(data=[go.Pie(
                labels=words,
                values=scores,
                hole=0.4,
                marker=dict(
                    colors=px.colors.qualitative.Set3,
                    line=dict(color='white', width=2)
                ),
                textinfo='label+percent',
                hovertemplate='<b>%{label}</b><br>分數: %{value:.4f}<br>佔比: %{percent}<extra></extra>'
            )])

            fig.update_layout(
                title=dict(
                    text=f'<b>{title}</b>',
                    x=0.5,
                    xanchor='center',
                    font=dict(size=20)
                ),
                showlegend=True,
                height=600,
                template='plotly_white',
                legend=dict(
                    orientation="v",
                    yanchor="middle",
                    y=0.5,
                    xanchor="left",
                    x=1.05
                )
            )

            print(f"✅ 圓餅圖生成完成")
            return fig

        except Exception as e:
            print(f"❌ 圓餅圖生成失敗: {str(e)}")
            return None

    def plot_methods_comparison(self, analysis_results, top_n=10):
        """比較不同分析方法"""
        print("📊 生成分析方法比較圖")

        try:
            methods = {
                'word_frequency': '詞頻統計',
                'jieba_tfidf': 'Jieba TF-IDF',
                'textrank': 'TextRank',
                'sklearn_tfidf': 'Sklearn TF-IDF'
            }

            all_keywords = set()
            for method_key in methods.keys():
                if method_key in analysis_results and analysis_results[method_key]:
                    for word, _ in analysis_results[method_key][:top_n]:
                        all_keywords.add(word)

            if not all_keywords:
                print("⚠️ 沒有關鍵字資料")
                return None

            data = []
            for method_key, method_name in methods.items():
                if method_key in analysis_results and analysis_results[method_key]:
                    keywords_dict = dict(analysis_results[method_key][:top_n])
                    for word in all_keywords:
                        score = float(keywords_dict.get(word, 0))
                        if score > 0:
                            data.append({
                                '方法': method_name,
                                '關鍵字': word,
                                '分數': score
                            })

            if not data:
                print("⚠️ 沒有有效資料")
                return None

            df = pd.DataFrame(data)

            fig = px.bar(
                df,
                x='分數',
                y='關鍵字',
                color='方法',
                barmode='group',
                title='<b>不同分析方法的關鍵字比較</b>',
                labels={'分數': '分數', '關鍵字': '關鍵字', '方法': '分析方法'},
                color_discrete_sequence=px.colors.qualitative.Set2,
                height=max(600, len(all_keywords) * 40)
            )

            fig.update_layout(
                template='plotly_white',
                xaxis_title='分數',
                yaxis_title='關鍵字',
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                margin=dict(l=120, r=50, t=100, b=50)
            )

            print(f"✅ 方法比較圖生成完成")
            return fig

        except Exception as e:
            print(f"❌ 方法比較圖生成失敗: {str(e)}")
            return None

    def plot_all_methods_heatmap(self, analysis_results, top_n=15):
        """熱力圖"""
        print("📊 生成分析方法熱力圖")

        try:
            methods = {
                'word_frequency': '詞頻統計',
                'jieba_tfidf': 'Jieba TF-IDF',
                'textrank': 'TextRank',
                'sklearn_tfidf': 'Sklearn TF-IDF'
            }

            all_keywords = set()
            for method_key in methods.keys():
                if method_key in analysis_results and analysis_results[method_key]:
                    for word, _ in analysis_results[method_key][:top_n]:
                        all_keywords.add(word)

            if not all_keywords:
                return None

            keywords_list = sorted(list(all_keywords))
            matrix = []
            method_names = []

            for method_key, method_name in methods.items():
                if method_key in analysis_results and analysis_results[method_key]:
                    method_names.append(method_name)
                    keywords_dict = dict(analysis_results[method_key])
                    row = [float(keywords_dict.get(kw, 0)) for kw in keywords_list]
                    matrix.append(row)

            if not matrix:
                return None

            fig = go.Figure(data=go.Heatmap(
                z=matrix,
                x=keywords_list,
                y=method_names,
                colorscale='Viridis',
                hovertemplate='方法: %{y}<br>關鍵字: %{x}<br>分數: %{z:.4f}<extra></extra>',
                colorbar=dict(title="分數")
            ))

            fig.update_layout(
                title='<b>關鍵字分析方法熱力圖</b>',
                xaxis_title='關鍵字',
                yaxis_title='分析方法',
                height=400,
                template='plotly_white',
                xaxis=dict(tickangle=-45)
            )

            print(f"✅ 熱力圖生成完成")
            return fig

        except Exception as e:
            print(f"❌ 熱力圖生成失敗: {str(e)}")
            return None

    def plot_keyword_network(self, keywords, top_n=20):
        """關鍵字網絡圖"""
        print("📊 生成關鍵字網絡圖")

        try:
            top_keywords = keywords[:top_n]

            if not top_keywords:
                print("⚠️ 沒有關鍵字資料")
                return None

            words = [kw for kw, _ in top_keywords]
            scores = [float(score) for _, score in top_keywords]

            max_score = max(scores)
            sizes = [s / max_score * 100 + 20 for s in scores]

            import random
            random.seed(42)

            x_coords = [random.uniform(0, 10) for _ in range(len(words))]
            y_coords = [random.uniform(0, 10) for _ in range(len(words))]

            fig = go.Figure(data=[go.Scatter(
                x=x_coords,
                y=y_coords,
                mode='markers+text',
                marker=dict(
                    size=sizes,
                    color=scores,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title="TF-IDF<br>分數"),
                    line=dict(width=2, color='white')
                ),
                text=words,
                textposition='middle center',
                textfont=dict(size=10, color='white'),
                hovertemplate='<b>%{text}</b><br>分數: %{marker.color:.4f}<extra></extra>'
            )])

            fig.update_layout(
                title='<b>關鍵字網絡圖（泡泡大小代表重要性）</b>',
                xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
                yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
                height=700,
                template='plotly_white',
                showlegend=False
            )

            print(f"✅ 網絡圖生成完成")
            return fig

        except Exception as e:
            print(f"❌ 網絡圖生成失敗: {str(e)}")
            return None

    def plot_news_timeline(self, news_data):
        """新聞時間軸"""
        print("📊 生成新聞時間軸")

        try:
            if not news_data:
                print("⚠️ 沒有新聞資料")
                return None

            df = pd.DataFrame(news_data)
            source_counts = df['來源'].value_counts()

            fig = go.Figure(data=[
                go.Bar(
                    x=source_counts.index,
                    y=source_counts.values,
                    marker=dict(
                        color=source_counts.values,
                        colorscale='Blues',
                        showscale=True,
                        colorbar=dict(title="新聞數量")
                    ),
                    text=source_counts.values,
                    textposition='outside',
                    hovertemplate='<b>%{x}</b><br>新聞數量: %{y}<extra></extra>'
                )
            ])

            fig.update_layout(
                title='<b>新聞來源分布</b>',
                xaxis_title='新聞來源',
                yaxis_title='新聞數量',
                height=400,
                template='plotly_white',
                showlegend=False
            )

            print(f"✅ 時間軸生成完成")
            return fig

        except Exception as e:
            print(f"❌ 時間軸生成失敗: {str(e)}")
            return None

    def create_comprehensive_dashboard(self, analysis_results, news_data):
        """建立綜合儀表板"""
        print("\n" + "="*60)
        print("🎨 開始生成視覺化儀表板")
        print("="*60)

        dashboard = {}

        # 1. 文字雲（傳統方式）
        if 'sklearn_tfidf' in analysis_results and analysis_results['sklearn_tfidf']:
            dashboard['wordcloud'] = self.create_wordcloud(
                analysis_results['sklearn_tfidf'],
                title="前50熱門關鍵字文字雲",
                max_words=50
            )

        # 2. Plotly 文字雲（備用方案）
        if 'sklearn_tfidf' in analysis_results and analysis_results['sklearn_tfidf']:
            dashboard['wordcloud_plotly'] = self.create_wordcloud_plotly(
                analysis_results['sklearn_tfidf'],
                title="關鍵字視覺化（Plotly版）",
                max_words=50
            )

        # 3. 關鍵字橫條圖
        if 'sklearn_tfidf' in analysis_results and analysis_results['sklearn_tfidf']:
            dashboard['bar_chart'] = self.plot_top_keywords_bar(
                analysis_results['sklearn_tfidf'],
                title="前20關鍵字排行榜",
                top_n=20
            )

        # 4. 圓餅圖
        if 'sklearn_tfidf' in analysis_results and analysis_results['sklearn_tfidf']:
            dashboard['pie_chart'] = self.plot_keyword_distribution_pie(
                analysis_results['sklearn_tfidf'],
                title="前15關鍵字分數分布",
                top_n=15
            )

        # 5. 方法比較圖
        dashboard['comparison'] = self.plot_methods_comparison(analysis_results, top_n=10)

        # 6. 熱力圖
        dashboard['heatmap'] = self.plot_all_methods_heatmap(analysis_results, top_n=15)

        # 7. 網絡圖
        if 'sklearn_tfidf' in analysis_results and analysis_results['sklearn_tfidf']:
            dashboard['network'] = self.plot_keyword_network(
                analysis_results['sklearn_tfidf'],
                top_n=20
            )

        # 8. 新聞時間軸
        if news_data:
            dashboard['timeline'] = self.plot_news_timeline(news_data)

        print("\n" + "="*60)
        print("✅ 視覺化儀表板生成完成")
        print("="*60 + "\n")

        return dashboard

In [26]:
# ==================== Google Sheets 操作 ====================
class SheetManager:
    """Google Sheets 管理"""

    def __init__(self, gc, sheet_url):
        self.gc = gc
        self.spreadsheet = gc.open_by_url(sheet_url)

    def write_news_data(self, news_data):
        """寫入新聞資料到 Sheet"""
        try:
            # 取得或建立工作表
            try:
                sheet = self.spreadsheet.worksheet('新聞資料')
            except:
                sheet = self.spreadsheet.add_worksheet(title='新聞資料', rows=1000, cols=10)

            # 清空並寫入標題
            sheet.clear()
            headers = ['標題', '摘要', '連結', '發布時間', '爬取時間', '來源']
            sheet.append_row(headers)

            # 寫入資料
            for news in news_data:
                row = [
                    news.get('標題', ''),
                    news.get('摘要', ''),
                    news.get('連結', ''),
                    news.get('發布時間', ''),
                    news.get('爬取時間', ''),
                    news.get('來源', '')
                ]
                sheet.append_row(row)

            return f"✅ 成功寫入 {len(news_data)} 則新聞到 Google Sheet"
        except Exception as e:
            return f"❌ 寫入失敗: {str(e)}"

    def read_news_data(self):
        """從 Sheet 讀取新聞資料"""
        try:
            sheet = self.spreadsheet.worksheet('新聞資料')
            data = sheet.get_all_records()
            return data
        except Exception as e:
            print(f"❌ 讀取失敗: {str(e)}")
            return []

    def write_keywords(self, keywords):
        """寫入關鍵字統計到 Sheet"""
        try:
            # 取得或建立工作表
            try:
                sheet = self.spreadsheet.worksheet('關鍵字統計')
            except:
                sheet = self.spreadsheet.add_worksheet(title='關鍵字統計', rows=100, cols=5)

            # 清空並寫入標題
            sheet.clear()
            headers = ['排名', '關鍵字', 'TF-IDF分數', '更新時間']
            sheet.append_row(headers)

            # 寫入資料
            for idx, (keyword, score) in enumerate(keywords, 1):
                row = [
                    idx,
                    keyword,
                    round(score, 4),
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                ]
                sheet.append_row(row)

            return f"✅ 成功寫入 {len(keywords)} 個關鍵字到統計表"
        except Exception as e:
            return f"❌ 寫入關鍵字失敗: {str(e)}"



In [19]:
# ==================== Gemini AI 分析 ====================
def generate_insights_with_gemini(model, news_data, keywords):
    """使用 Gemini 生成洞察摘要"""
    try:
        # 準備提示詞
        keywords_text = ', '.join([kw for kw, _ in keywords[:10]])
        news_titles = '\n'.join([f"- {news['標題']}" for news in news_data[:20]])

        prompt = f"""
你是專業的金融分析師。根據以下台股新聞資料，請提供分析：

【新聞標題】（共 {len(news_data)} 則）
{news_titles}

【關鍵字】
{keywords_text}

請提供：
1. **5 句關鍵洞察**（每句 30 字內，分點條列）
2. **總結論**（120 字，深入分析市場趨勢與投資建議）

格式如下：
## 關鍵洞察
1. [洞察1]
2. [洞察2]
3. [洞察3]
4. [洞察4]
5. [洞察5]

## 總結
[120字總結]
"""

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"❌ Gemini 分析失敗: {str(e)}\n\n請檢查 API Key 是否正確"

In [35]:
# ==================== 主流程整合（加入視覺化）====================
def automated_pipeline(sheet_url, gemini_api_key, crawl_pages=3, analysis_method='comprehensive'):
    """完整自動化流程 - 加入視覺化"""

    results = {
        'status': [],
        'news_data': None,
        'keywords': None,
        'analysis_details': None,
        'insights': None,
        'visualizations': None
    }

    try:
        # Step 1: 爬取新聞
        results['status'].append("🔍 步驟1: 開始爬取財經新聞...")
        crawler = FinanceNewsCrawler()
        news_data = crawler.auto_crawl(max_news=min(30, crawl_pages * 10))

        if not news_data:
            results['status'].append("❌ 爬取失敗，請檢查網路連線")
            return results

        results['news_data'] = news_data
        results['status'].append(f"✅ 成功爬取 {len(news_data)} 則新聞")

        # Step 2: 寫入 Google Sheet
        results['status'].append("📝 步驟2: 寫入 Google Sheet...")
        gc = setup_google_sheets()
        sheet_manager = SheetManager(gc, sheet_url)
        write_result = sheet_manager.write_news_data(news_data)
        results['status'].append(write_result)

        # Step 3: 使用結巴分詞進行文字分析
        results['status'].append("🔬 步驟3: 使用結巴分詞進行文字分析...")
        analyzer = TextAnalyzer()

        # 合併標題和摘要
        texts = [f"{n['標題']} {n.get('摘要', '')}" for n in news_data]

        # 執行綜合分析
        if analysis_method == 'comprehensive':
            results['status'].append("📊 執行綜合分析（詞頻 + TF-IDF + TextRank）...")
            analysis_results = analyzer.comprehensive_analysis(texts, top_n=50)  # 提取前50個
            results['analysis_details'] = analysis_results

            # 使用 TF-IDF 作為主要關鍵字
            keywords = analysis_results.get('sklearn_tfidf', [])
            if not keywords:
                keywords = analysis_results.get('jieba_tfidf', [])
        else:
            keywords = analyzer.get_best_keywords(texts, top_n=50, method=analysis_method)

        results['keywords'] = keywords
        results['status'].append(f"✅ 提取前 50 個關鍵字")

        # Step 4: 回寫關鍵字到統計表
        results['status'].append("📊 步驟4: 回寫關鍵字統計...")
        keyword_result = sheet_manager.write_keywords(keywords[:10])  # 只寫入前10個
        results['status'].append(keyword_result)

        # Step 4.5: 回寫詳細分析結果
        if analysis_method == 'comprehensive' and results['analysis_details']:
            results['status'].append("📊 步驟4.5: 回寫詳細分析結果...")
            try:
                sheet_manager.write_comprehensive_analysis(results['analysis_details'])
                results['status'].append("✅ 詳細分析結果已寫入")
            except Exception as e:
                results['status'].append(f"⚠️ 詳細分析寫入失敗: {str(e)}")

        # Step 5: 生成視覺化圖表
        results['status'].append("🎨 步驟5: 生成視覺化圖表...")
        visualizer = DataVisualizer()

        if analysis_method == 'comprehensive' and results['analysis_details']:
            visualizations = visualizer.create_comprehensive_dashboard(
                results['analysis_details'],
                news_data
            )
        else:
            # 簡化版視覺化
            visualizations = {
                'wordcloud': visualizer.create_wordcloud(keywords, max_words=50),
                'bar_chart': visualizer.plot_top_keywords_bar(keywords, top_n=20),
                'pie_chart': visualizer.plot_keyword_distribution_pie(keywords, top_n=15),
                'network': visualizer.plot_keyword_network(keywords, top_n=20),
                'timeline': visualizer.plot_news_timeline(news_data)
            }

        results['visualizations'] = visualizations
        results['status'].append("✅ 視覺化圖表生成完成")

        # Step 6: 使用 Gemini 生成洞察
        if gemini_api_key and gemini_api_key.strip():
            results['status'].append("🤖 步驟6: 使用 Gemini AI 生成洞察...")
            try:
                model = setup_gemini(gemini_api_key)
                insights = generate_insights_with_gemini(model, news_data, keywords[:10], results.get('analysis_details'))
                results['insights'] = insights
                results['status'].append("✅ Gemini 分析完成")
            except Exception as e:
                results['insights'] = f"❌ Gemini 分析失敗: {str(e)}\n請檢查 API Key 是否正確"
                results['status'].append("⚠️ Gemini 分析失敗（可能是 API Key 錯誤）")
        else:
            results['insights'] = "⚠️ 未提供 Gemini API Key，跳過 AI 分析"
            results['status'].append("⚠️ 跳過 Gemini 分析（未提供 API Key）")

        results['status'].append("\n🎉 所有步驟完成！")

    except Exception as e:
        results['status'].append(f"❌ 流程錯誤: {str(e)}")

    return results

In [42]:
# ==================== Gradio 介面（文字雲修正版）====================
def create_gradio_interface():
    """建立 Gradio 介面 - 文字雲修正版"""

    def run_pipeline(sheet_url, gemini_key, pages):
        """執行完整流程"""
        results = automated_pipeline(sheet_url, gemini_key, int(pages), 'comprehensive')

        # 狀態訊息
        status_text = '\n'.join(results['status'])

        # 關鍵字表格
        if results['keywords']:
            keywords_df = pd.DataFrame(
                results['keywords'][:20],
                columns=['關鍵字', 'TF-IDF分數']
            )
            keywords_df.index = range(1, len(keywords_df) + 1)
            keywords_df['TF-IDF分數'] = keywords_df['TF-IDF分數'].round(4)
        else:
            keywords_df = pd.DataFrame()

        # 新聞表格
        if results['news_data']:
            news_df = pd.DataFrame(results['news_data'])
            news_df = news_df[['標題', '發布時間', '來源', '連結']]
        else:
            news_df = pd.DataFrame()

        # Gemini 洞察
        insights_text = results['insights'] if results['insights'] else "尚未生成分析"

        # 視覺化圖表
        viz = results.get('visualizations', {})

        wordcloud_img = viz.get('wordcloud')  # 傳統文字雲（圖片）
        wordcloud_plotly = viz.get('wordcloud_plotly')  # Plotly文字雲（互動圖表）
        bar_chart = viz.get('bar_chart')
        pie_chart = viz.get('pie_chart')
        comparison_chart = viz.get('comparison')
        heatmap_chart = viz.get('heatmap')
        network_chart = viz.get('network')
        timeline_chart = viz.get('timeline')

        return (
            status_text,
            keywords_df,
            news_df,
            insights_text,
            wordcloud_img,  # 傳統文字雲
            wordcloud_plotly,  # Plotly文字雲
            bar_chart,
            pie_chart,
            comparison_chart,
            heatmap_chart,
            network_chart,
            timeline_chart
        )

    # 建立介面
    with gr.Blocks(title="🚀 台股財經新聞分析系統", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🚀 台股財經新聞自動化分析系統
        ### 📊 爬蟲 → Google Sheet → 結巴分詞 → TF-IDF → Gemini AI → 視覺化儀表板
        """)

        with gr.Row():
            with gr.Column(scale=2):
                sheet_url_input = gr.Textbox(
                    label="📄 Google Sheet 網址",
                    placeholder="https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit",
                    value="https://docs.google.com/spreadsheets/d/1UzNgDMKD_WH3uhfQXdrBPM_z8oFHLh77yahMbu5KlHo/edit?usp=sharing"
                )
            with gr.Column(scale=2):
                gemini_key_input = gr.Textbox(
                    label="🔑 Gemini API Key (選填)",
                    placeholder="輸入你的 Gemini API Key 或留空跳過 AI 分析",
                    type="password"
                )
            with gr.Column(scale=1):
                pages_input = gr.Number(
                    label="📄 爬取頁數",
                    value=3,
                    minimum=1,
                    maximum=5
                )

        run_button = gr.Button("🚀 開始執行完整流程", variant="primary", size="lg")

        gr.Markdown("---")

        with gr.Tabs():
            with gr.Tab("📋 執行狀態"):
                status_output = gr.Textbox(
                    label="流程狀態",
                    lines=20,
                    interactive=False
                )

            with gr.Tab("🔑 關鍵字分析"):
                gr.Markdown("### 📊 前 20 熱門關鍵字 (TF-IDF 分析)")
                keywords_output = gr.Dataframe(
                    label="關鍵字排行",
                    headers=['關鍵字', 'TF-IDF分數'],
                    interactive=False
                )

            with gr.Tab("📰 新聞列表"):
                gr.Markdown("### 📰 爬取的新聞資料")
                news_output = gr.Dataframe(
                    label="新聞列表",
                    headers=['標題', '發布時間', '來源', '連結'],
                    interactive=False,
                    wrap=True
                )

            with gr.Tab("🤖 AI 洞察"):
                gr.Markdown("### 🤖 Gemini AI 深度分析")
                insights_output = gr.Markdown(value="等待執行...")

            with gr.Tab("☁️ 文字雲（傳統）"):
                gr.Markdown("### ☁️ 前50熱門關鍵字文字雲（WordCloud）")
                gr.Markdown("使用 WordCloud 套件生成，字體大小代表重要性")
                wordcloud_output = gr.Image(label="關鍵字文字雲", type="filepath")

            with gr.Tab("☁️ 文字雲（互動）"):
                gr.Markdown("### ☁️ 關鍵字視覺化（Plotly 互動版）")
                gr.Markdown("可縮放、拖曳、懸停查看分數")
                wordcloud_plotly_output = gr.Plot(label="Plotly 文字雲")

            with gr.Tab("📊 橫條圖"):
                gr.Markdown("### 📊 前 20 關鍵字排行榜")
                bar_output = gr.Plot(label="關鍵字橫條圖")

            with gr.Tab("🥧 圓餅圖"):
                gr.Markdown("### 🥧 關鍵字分數分布")
                pie_output = gr.Plot(label="關鍵字圓餅圖")

            with gr.Tab("🔀 方法比較"):
                gr.Markdown("### 🔀 不同分析方法的關鍵字比較")
                comparison_output = gr.Plot(label="分析方法比較")

            with gr.Tab("🔥 熱力圖"):
                gr.Markdown("### 🔥 關鍵字分析熱力圖")
                heatmap_output = gr.Plot(label="熱力圖")

            with gr.Tab("🕸️ 網絡圖"):
                gr.Markdown("### 🕸️ 關鍵字網絡圖（泡泡圖）")
                network_output = gr.Plot(label="關鍵字網絡圖")

            with gr.Tab("📈 新聞分布"):
                gr.Markdown("### 📈 新聞來源分布統計")
                timeline_output = gr.Plot(label="新聞來源統計")

        # 綁定按鈕
        run_button.click(
            run_pipeline,
            inputs=[sheet_url_input, gemini_key_input, pages_input],
            outputs=[
                status_output,
                keywords_output,
                news_output,
                insights_output,
                wordcloud_output,  # 傳統文字雲
                wordcloud_plotly_output,  # Plotly文字雲
                bar_output,
                pie_output,
                comparison_output,
                heatmap_output,
                network_output,
                timeline_output
            ]
        )

        gr.Markdown("""
        ---
        ### 💡 使用說明

        現在提供**兩種文字雲**：
        1. **☁️ 傳統文字雲**: 使用 WordCloud 套件，視覺效果最佳
        2. ☁️ 互動文字雲: 使用 Plotly，可縮放、拖曳、懸停

        #### 🎨 視覺化圖表
        - ☁️ **文字雲（兩種）**: 前50個關鍵字視覺化
        - 📊 **橫條圖**: 前20個關鍵字排行
        - 🥧 **圓餅圖**: 前15個關鍵字佔比
        - 🔀 **方法比較**: 4種分析方法對比
        - 🔥 **熱力圖**: 關鍵字×方法矩陣
        - 🕸️ **網絡圖**: 泡泡大小=重要性
        - 📈 **新聞分布**: 來源統計
        """)

        return demo

In [ ]:
# ==================== 啟動應用 ====================
if __name__ == "__main__":
    print("🚀 正在啟動台股財經新聞分析系統...")
    print("📊 包含完整視覺化儀表板")
    print("=" * 60)
    app = create_gradio_interface()
    app.launch(share=True, debug=True)

🚀 正在啟動台股財經新聞分析系統...
📊 包含完整視覺化儀表板
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4ac96dcf9f5151531a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🚀 開始智慧爬蟲...
📡 嘗試使用: RSS Feed
🔍 正在爬取聯合新聞網 RSS...
✅ 成功爬取 30 則新聞（聯合新聞網）
✅ 成功！使用 RSS Feed 取得 30 則新聞
🔧 初始化結巴分詞...
✅ 結巴分詞初始化完成
📚 停用詞數量: 136 個
📖 自訂詞彙數量: 98 個

🚀 開始綜合文字分析（已過濾網址和雜訊）

📊 方法1: 詞頻統計
🔍 進行詞頻統計...
✅ 統計了 137 個不同的詞，返回前 50 個

【詞頻統計】前 10 個關鍵字:
  1. img (23.0000)
  2. src (23.0000)
  3. 台積電 (10.0000)
  4. 聯準會 (3.0000)
  5. Fed (3.0000)
  6. 半導體 (3.0000)
  7. 人壽成 (2.0000)
  8. 贊助夥伴 (2.0000)
  9. TPVL (2.0000)
  10. 徵台灣 (2.0000)

📊 方法2: jieba TF-IDF
🔍 使用 jieba.analyse 提取關鍵字...
✅ 提取了 9 個有效關鍵字

【jieba TF-IDF】前 10 個關鍵字:
  1. 聯邦銀行 (0.0373)
  2. 記憶體 (0.0373)
  3. 杜金龍 (0.0373)
  4. 陶朱隱 (0.0373)
  5. 黃立成 (0.0373)
  6. 伺服器 (0.0189)
  7. 內政部 (0.0187)
  8. 停電降 (0.0187)
  9. 指數續 (0.0187)

📊 方法3: TextRank 演算法
🔍 使用 TextRank 演算法提取關鍵字...
✅ 提取了 7 個有效關鍵字

【TextRank】前 10 個關鍵字:
  1. 陶朱隱 (0.4135)
  2. 黃立成 (0.3885)
  3. 記憶體 (0.3666)
  4. 概念股 (0.2595)
  5. 內政部 (0.2541)
  6. 信義區 (0.2336)
  7. 杜金龍 (0.2318)

📊 方法4: sklearn TF-IDF
🔍 使用 TF-IDF 分析關鍵字...
✅ TF-IDF 分析完成，提取 50 個關鍵字

【sklearn TF-IDF】前 10 個關鍵字:
  1. img (3.6

Traceback (most recent call last):
  File "/tmp/ipython-input-3580048705.py", line 45, in create_wordcloud
    ).generate_from_frequencies(word_freq)
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wordcloud/wordcloud.py", line 453, in generate_from_frequencies
    self.generate_from_frequencies(dict(frequencies[:2]),
  File "/usr/local/lib/python3.12/dist-packages/wordcloud/wordcloud.py", line 506, in generate_from_frequencies
    font = ImageFont.truetype(self.font_path, font_size)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/PIL/ImageFont.py", line 880, in truetype
    return freetype(font)
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/PIL/ImageFont.py", line 877, in freetype
    return FreeTypeFont(font, size, index, encoding, layout_engine)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packag

✅ 網絡圖生成完成
📊 生成新聞時間軸
✅ 時間軸生成完成

✅ 視覺化儀表板生成完成

